![NYC Skyline](nyc.jpg)

Welcome to New York City, one of the most-visited cities in the world. There are many Airbnb listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this project, we will take a closer look at the New York Airbnb market by combining data from multiple file types like `.csv`, `.tsv`, and `.xlsx`.

Recall that **CSV**, **TSV**, and **Excel** files are three common formats for storing data. 
Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv**
This is a CSV file containing data on Airbnb listing prices and locations.
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed

## Importing packages and dataset(s)
* Importing `numpy` and `pandas`
* Importing datasets, `price`, `room_type`, and `last_review`

In [3]:
# Import packages
import numpy as np
import pandas as pd

# Import dataset(s)
price = pd.read_csv("data/airbnb_price.csv")
room_type = pd.read_excel("data/airbnb_room_type.xlsx")
last_review = pd.read_csv("data/airbnb_last_review.tsv", sep='\t')

In [4]:
# Confirming correct dataset load
datasets = [price, room_type, last_review]

for dataset in datasets:
    print(f"Memory usage: {dataset.memory_usage().sum()}")
    print(dataset.head(5))
    print(f"")
    print({dataset.shape})

Memory usage: 605144
   listing_id        price                nbhood_full
0        2595  225 dollars         Manhattan, Midtown
1        3831   89 dollars     Brooklyn, Clinton Hill
2        5099  200 dollars     Manhattan, Murray Hill
3        5178   79 dollars  Manhattan, Hell's Kitchen
4        5238  150 dollars       Manhattan, Chinatown

{(25209, 3)}
Memory usage: 605144
   listing_id                                description        room_type
0        2595                      Skylit Midtown Castle  Entire home/apt
1        3831            Cozy Entire Floor of Brownstone  Entire home/apt
2        5099  Large Cozy 1 BR Apartment In Midtown East  Entire home/apt
3        5178            Large Furnished Room Near B'way     private room
4        5238         Cute & Cozy Lower East Side 1 bdrm  Entire home/apt

{(25209, 3)}
Memory usage: 605144
   listing_id    host_name   last_review
0        2595     Jennifer   May 21 2019
1        3831  LisaRoxanne  July 05 2019
2        5099     

## Merging dataframes
* Join `price` with `room_type` on `listing_id` , concurrently initialize new dataframe `df`
* Join `last_review` on `room_type` on `listing_id`

In [5]:
# Initialize new df, join price and room_type dataframes on listing_id
df = pd.merge(left=room_type, right=price, on="listing_id")

In [6]:
# Jpin last_reivew and room_type on listing_id
df = df.merge(last_review, on="listing_id")

## First review date and last review date
* If needed, clean data to have date-time objects
* Select min date in `df`
* Select max date in `df`

In [7]:
# Check data types 
df.dtypes

listing_id      int64
description    object
room_type      object
price          object
nbhood_full    object
host_name      object
last_review    object
dtype: object

In [8]:
# Set last_review["last_review"] to datetime object
df["last_review"] = pd.to_datetime(df["last_review"])

# Verify if last_review["last_review"] is set to datetime
print(df["last_review"].dtype)

# Show head of dataset
print(df["last_review"].head())

datetime64[ns]
0   2019-05-21
1   2019-07-05
2   2019-06-22
3   2019-06-24
4   2019-06-09
Name: last_review, dtype: datetime64[ns]


In [9]:
# Select the min date of the most recent reviews
min_review_date = df["last_review"].min()
print(f"The oldest Airbnb review is from {min_review_date}")

# Select the max date of the most recent reviews
max_review_date = df["last_review"].max()
print(f"The newest Airbnb review is from {max_review_date}")

The oldest Airbnb review is from 2019-01-01 00:00:00
The newest Airbnb review is from 2019-07-09 00:00:00


## How many of the listings are private rooms?
* Clean data
* Subset private rooms
* Count rows of private rooms
* Save count in a variable called `total_private_rooms`

In [10]:
# Cleaning df["room_type"]
df["room_type"] = df["room_type"].str.lower()

# Subset private rooms
private_listings = df[df["room_type"] == "private room"]

# Counts amount of private rooms and stores it in the value total_private_rooms
total_private_rooms = np.sum(private_listings.value_counts(dropna=False))

## What is the average listing price?
* Clean data
* Calculate average listing price
* Save calculation as `avg_listing_price`

In [11]:
# Clear all string data
df["price"] = df["price"].str.replace(" dollars", "")
df["price"] = df["price"].str.strip()

# Verify if Series is clean of string data
print(df["price"].head(5))

# Set category as int64
df["price"] = df["price"].astype(int)

# Verify datatype of df["price"] as int64
print(df["price"].dtype)

0    225
1     89
2    200
3     79
4    150
Name: price, dtype: object
int64


In [12]:
# Calculate average price and store the value
avg_listing_price = np.round(np.average(df["price"]), 2)

## Bringing it all together
* Storing first_reviewed, last_reviewed, nb_private_rooms and avg_price as a dictionary
* Converting dictionary to dataframe

In [13]:
dict_review_dates = {
    "first_reviewed": [min_review_date],
    "last_reviewed": [max_review_date],
    "nb_private_rooms": [total_private_rooms],
    "avg_price": [avg_listing_price],
}

review_dates = pd.DataFrame(dict_review_dates)